# 情绪投资策略(2014-2022年全A股市场)

## 1.数据准备

In [1]:
import sys
import os
import numpy as np
import cudf  #CUDA计算
import pandas as pd

sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')
from pystata import config  #Stata
from statsmodels.regression.rolling import RollingOLS  #滚动回归
from pandarallel import pandarallel  #多线程groupby Apply

config.init('mp')

# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = './DataSets/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


### 1.1 下载和合并面板数据

In [2]:
%%time

# 下载数据
def load_data():
    # 数据集:个股K线面板数据,个股基本面数据
    data_list = ['ASHARE_BAR_PANEL.parquet', 'ASHARE_BASIC_PANEL.parquet']
    from loader.findata_loader import DownLoader
    if not set(data_list).issubset(os.listdir('./DataSets/')): DownLoader(MAX_CORE=10).load_data()


load_data()

CPU times: user 80.2 ms, sys: 2.35 ms, total: 82.6 ms
Wall time: 87.2 ms


In [3]:
%%time

# 提取面板数据
def extract_panel():
    # 个股K线数据
    df_bar = (
        cudf.read_parquet('./DataSets/ASHARE_BAR_PANEL.parquet', columns=['trade_date', 'ts_code', 'pct_chg'])
        .rename(columns={'pct_chg': 'share_return'})
    )

    # 个股基本面数据
    df_basic = cudf.read_parquet('./DataSets/ASHARE_BASIC_PANEL.parquet', columns=['trade_date', 'ts_code', 'total_mv'])

    # 合并
    df_p = cudf.concat([df_bar, df_basic], join="left", axis=1, sort=True)

    # 压缩数据
    df_p.index.levels[1].astype('category', inplace=True)

    return df_p


# 提取时间序列数据
def extract_time_series():
    # 股指数据
    from utils.sql import DB
    db_loader = DB()
    df_share_index = (
        pd.read_sql_table('399300.SZ', db_loader.ENGINE, 'FIN_DAILY_INDEX', columns=['trade_date', 'pct_chg'])
        .astype(dtype={'trade_date': 'uint32'}).set_index('trade_date').rename(columns={'pct_chg': 'shareindex_return'})
    )

    # shibor数据
    df_shibor = (
            pd.read_sql_table('SHIBOR', db_loader.ENGINE, 'FIN_DAILY_INDEX', columns=['trade_date', '3m'])
            .astype(dtype={'trade_date': 'uint32'}).set_index('trade_date').rename(columns={'3m': 'riskfree_return'})/360
    )

    # 情绪数据
    df_sent = pd.concat(
            [pd.read_sql_table('IMG_SENT', db_loader.ENGINE, 'SENT_DAILY').astype(dtype={'trade_date': 'uint32'})
             .set_index('trade_date').rename(columns={'neg_index': 'img_neg'}),
             pd.read_sql_table('TEX_SENT', db_loader.ENGINE, 'SENT_DAILY').astype(dtype={'trade_date': 'uint32'})
             .set_index('trade_date').rename(columns={'neg_index': 'tex_neg'})
             ], axis=1
    )

    return (cudf.from_pandas(pd.concat([df_share_index, df_shibor], join="inner", axis=1, sort=True)),
            pd.concat([df_sent, df_share_index, df_shibor], join="inner", axis=1, sort=True))


# 合并数据
def extract_merge():
    df_p = extract_panel()
    df_t, _ = extract_time_series()
    df_m = cudf.merge(
            left=df_p.reset_index(), right=df_t.reset_index(), left_on='trade_date', right_on='trade_date', how="left",
            sort=True
    )
    return df_m.set_index(['trade_date', 'ts_code']).sort_index(ascending=[True, True])


df_panel = extract_merge()
_, df_time_series = extract_time_series()

# 数据筛选
df_panel = df_panel[df_panel.index.get_level_values('trade_date') >= 20140101].to_pandas()
df_panel

CPU times: user 2.99 s, sys: 1.63 s, total: 4.62 s
Wall time: 4.63 s


share_return      total_mv  shareindex_return  \
trade_date ts_code                                                    
20140102   000001.SZ       -0.1641  1.002537e+07            -0.3454   
           000002.SZ       -0.4972  8.799966e+06            -0.3454   
           000004.SZ        1.3734  9.917646e+04            -0.3454   
           000005.SZ       -0.4000  2.276691e+05            -0.3454   
           000006.SZ       -1.2164  6.574476e+05            -0.3454   
...                            ...           ...                ...   
20221130   872374.BJ       -1.7259           NaN             0.1199   
20221201   301290.SZ       -8.7349  4.856403e+05             1.0831   
           301311.SZ       12.7436  5.414400e+05             1.0831   
           870199.BJ       -3.1447  1.498420e+05             1.0831   
           872374.BJ        0.2066           NaN             1.0831   

                      riskfree_return  
trade_date ts_code                     
20140102   000001.SZ         0.015460  
           000002.SZ         0.015460  
           000004.SZ         0.015460  
           000005.SZ         0.015460  
           000006.SZ         0.015460  
...                               ...  
20221130   872374.BJ         0.006092  
20221201   301290.SZ         0.006103  
           301311.SZ         0.006103  
           870199.BJ         0.006103  
           872374.BJ         0.006103  

[7143120 rows x 4 columns]

## 2.构造截面异质波动率与市值高低组合

#### 2.1 计算面板数据的异质波动率IDVOL

In [4]:
%%time

# 滚动OLS回归求异质波动率
def roll_idvol(df_code: pd.DataFrame, ols_window: int, var_ma: int) -> pd.DataFrame:
    try:
        # 估计参数
        model_ols = RollingOLS(endog=df_code[['Y']], exog=df_code[['CONST', 'X']], window=ols_window)
        df_para = model_ols.fit().params.rename(columns={'CONST': 'Alpha', 'X': 'Beta'})

        # 预测残差 已经对齐了
        df_con = pd.concat([df_code, df_para], axis=1, join='inner')
        df_con['Residual'] = df_con['Alpha'] + df_con['Beta']*df_con['X'] - df_con['Y']

        # 计算月波动率
        df_con['Idvol'] = df_con['Residual'].rolling(var_ma).var(ddof=1)
        return df_con[['share_return', 'total_mv', 'Idvol']]
    except (IndexError, ValueError): return pd.DataFrame(columns=['trade_date', 'ts_code']).set_index(['trade_date', 'ts_code'])


# 分组计算
def cal_panel_ols():
    # 定义回归变量 CAPM回归: (rm-rf)=a+b*(RM-rf)
    df_panel['Y'] = df_panel['share_return'] - df_panel['riskfree_return']
    df_panel['CONST'] = 1  # 带截距项回归
    df_panel['X'] = df_panel['shareindex_return'] - df_panel['riskfree_return']

    # 多线程加速
    pandarallel.initialize(progress_bar=True)
    df_out = (df_panel.groupby(level=['ts_code'])[['share_return', 'total_mv', 'Y', 'CONST', 'X']]
              .parallel_apply(lambda x: roll_idvol(x, 5, 30)).droplevel(2)
              )

    # 保存
    df_out.to_parquet(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet', engine='pyarrow', index=True)


# 计算滚动回归
if not os.path.exists(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet'): cal_panel_ols()

# 加载滚动回归的面板数据集
df_ols_panel = (
    cudf.read_parquet(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet').reset_index().set_index(['trade_date', 'ts_code']).sort_index()
)
df_ols_panel

CPU times: user 121 ms, sys: 80.8 ms, total: 202 ms
Wall time: 200 ms


share_return      total_mv        Idvol
trade_date ts_code                                           
20140102   000001.SZ       -0.1641  1.002537e+07         <NA>
           000002.SZ       -0.4972  8.799966e+06         <NA>
           000004.SZ        1.3734  9.917646e+04         <NA>
           000005.SZ       -0.4000  2.276691e+05         <NA>
           000006.SZ       -1.2164  6.574476e+05         <NA>
...                            ...           ...          ...
20221128   873122.BJ       -2.0158  1.259199e+05  19.27363078
           873169.BJ       -1.5198  5.443271e+04  1.319910083
           873223.BJ       -0.2660  5.898188e+04  0.730762806
           873339.BJ       -0.5093  1.290394e+05         <NA>
           873527.BJ        0.0000  5.445873e+04         <NA>

[7136670 rows x 3 columns]

### 2.2 面板数据异质波动率分组

In [5]:
%%time

# 参数阈值
QUANTILE = 0.5


# 面板数据分组
def group_ols_panel(df):
    # 分组
    df['idvol_top'] = df['Idvol'].groupby(level=['trade_date']).transform(lambda x: x.quantile(QUANTILE))
    df['idvol_group'] = np.where(df['Idvol'].to_pandas() >= df['idvol_top'].to_pandas(), "HIGH", "LOW")
    df = df.reset_index().set_index(['trade_date', 'idvol_group', 'ts_code']).sort_index()

    # 求组中市值加权系数,并求回报
    df['mv_ratio'] = df['total_mv']/df.groupby(level=['trade_date', 'idvol_group'])['total_mv'].transform('sum')

    # 求组中回报
    df['idvol_vw_ratio'] = df['mv_ratio']*df['share_return']
    df['idvol_group_return'] = (df.groupby(level=['trade_date', 'idvol_group'])['idvol_vw_ratio'].transform('sum'))

    return df


df_group_panel = group_ols_panel(df_ols_panel)
df_group_panel

CPU times: user 3 s, sys: 1.04 s, total: 4.04 s
Wall time: 4.04 s


share_return      total_mv        Idvol  \
trade_date idvol_group ts_code                                              
20140102   LOW         000001.SZ       -0.1641  1.002537e+07         <NA>   
                       000002.SZ       -0.4972  8.799966e+06         <NA>   
                       000004.SZ        1.3734  9.917646e+04         <NA>   
                       000005.SZ       -0.4000  2.276691e+05         <NA>   
                       000006.SZ       -1.2164  6.574476e+05         <NA>   
...                                        ...           ...          ...   
20221128   LOW         872925.BJ       -0.3987  7.386323e+04  1.366920583   
                       873169.BJ       -1.5198  5.443271e+04  1.319910083   
                       873223.BJ       -0.2660  5.898188e+04  0.730762806   
                       873339.BJ       -0.5093  1.290394e+05         <NA>   
                       873527.BJ        0.0000  5.445873e+04         <NA>   

                                    idvol_top  mv_ratio  idvol_vw_ratio  \
trade_date idvol_group ts_code                                            
20140102   LOW         000001.SZ         <NA>  0.003834       -0.000629   
                       000002.SZ         <NA>  0.003365       -0.001673   
                       000004.SZ         <NA>  0.000038        0.000052   
                       000005.SZ         <NA>  0.000087       -0.000035   
                       000006.SZ         <NA>  0.000251       -0.000306   
...                                       ...       ...             ...   
20221128   LOW         872925.BJ  2.944717594  0.000013       -0.000005   
                       873169.BJ  2.944717594  0.000010       -0.000015   
                       873223.BJ  2.944717594  0.000011       -0.000003   
                       873339.BJ  2.944717594  0.000023       -0.000012   
                       873527.BJ  2.944717594  0.000010        0.000000   

                                  idvol_group_return  
trade_date idvol_group ts_code                        
20140102   LOW         000001.SZ            0.104007  
                       000002.SZ            0.104007  
                       000004.SZ            0.104007  
                       000005.SZ            0.104007  
                       000006.SZ            0.104007  
...                                              ...  
20221128   LOW         872925.BJ           -0.769940  
                       873169.BJ           -0.769940  
                       873223.BJ           -0.769940  
                       873339.BJ           -0.769940  
                       873527.BJ           -0.769940  

[7136670 rows x 7 columns]

In [6]:
# 转为时间序列数据
df_series = (
    df_group_panel[['idvol_group_return']].groupby(level=['trade_date', 'idvol_group']).first().reset_index()
    .pivot(index='trade_date', columns='idvol_group', values='idvol_group_return').to_pandas()
)

# 合并其他时间序列数据
df_series_join = df_time_series.join(df_series).dropna(axis=0).reset_index()
df_series_join

,trade_date,img_neg,tex_neg,shareindex_return,riskfree_return,HIGH,LOW
0,20140225,0.000000,0.000000,-2.5559,0.015440,-4.016731,-1.827917
1,20140226,0.666667,0.000000,0.2547,0.015393,0.985354,0.193332
2,20140227,0.000000,0.000000,-0.4297,0.015378,-0.781264,0.026462
3,20140228,0.333333,0.666667,1.1542,0.015358,1.174221,0.582117
4,20140303,0.333333,0.333333,0.5231,0.015331,1.989484,0.722860
...,...,...,...,...,...,...,...
2026,20220621,0.238095,0.285714,-0.1124,0.005556,-0.810936,0.125217
2027,20220622,0.227273,0.227273,-1.2702,0.005556,-0.807400,-1.345733
2028,20220623,0.363636,0.363636,1.7153,0.005556,2.629145,1.459178
2029,20220624,0.200000,0.300000,1.1716,0.005556,1.323591,0.901009


In [32]:
pd.concat([df_time_series,df_series],axis=0)

,img_neg,tex_neg,shareindex_return,riskfree_return,HIGH,LOW
trade_date,,,,,,
20140102,0.000000,0.000000,-0.3454,0.015460,NaN,0.104007
20140103,0.000000,0.333333,-1.3436,0.015461,NaN,-0.901783
20140106,0.285714,0.142857,-2.2762,0.015481,NaN,-1.873938
20140107,0.000000,0.333333,-0.0284,0.015489,NaN,0.194875
20140108,0.333333,0.000000,0.1747,0.015506,NaN,0.194233
...,...,...,...,...,...,...
20221122,NaN,NaN,NaN,NaN,-1.561920,0.606623
20221123,NaN,NaN,NaN,NaN,-0.214574,0.214863
20221124,NaN,NaN,NaN,NaN,0.347296,-0.313833


## 3.VAR模型分析

### 3.1 回归前数据处理

In [13]:
# 增加平方项
def add_square_column(df, square_column: list): return pd.concat([df, df[square_column].pow(2).add_suffix('_s')], axis=1)


# 增加日期虚拟变量
def add_dummy_column(df, dummy_column: str):
    df_weekday = pd.get_dummies(pd.to_datetime(df[dummy_column], format='%Y%m%d').dt.weekday, prefix='weekday', drop_first=True)
    df_month = pd.get_dummies(pd.to_datetime(df[dummy_column], format='%Y%m%d').dt.month, prefix='month', drop_first=True)
    return pd.concat([df, df_weekday, df_month], axis=1)


df_series_ols = add_dummy_column(add_square_column(df_series_join, ['shareindex_return','HIGH','LOW']), 'trade_date')
df_series_ols
# @formatter:off

,trade_date,img_neg,tex_neg,shareindex_return,riskfree_return,HIGH,LOW,shareindex_return_s,HIGH_s,LOW_s,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,20140225,0.000000,0.000000,-2.5559,0.015440,-4.016731,-1.827917,6.532625,16.134125,3.341279,...,0,0,0,0,0,0,0,0,0,0
1,20140226,0.666667,0.000000,0.2547,0.015393,0.985354,0.193332,0.064872,0.970923,0.037377,...,0,0,0,0,0,0,0,0,0,0
2,20140227,0.000000,0.000000,-0.4297,0.015378,-0.781264,0.026462,0.184642,0.610373,0.000700,...,0,0,0,0,0,0,0,0,0,0
3,20140228,0.333333,0.666667,1.1542,0.015358,1.174221,0.582117,1.332178,1.378796,0.338860,...,0,0,0,0,0,0,0,0,0,0
4,20140303,0.333333,0.333333,0.5231,0.015331,1.989484,0.722860,0.273634,3.958047,0.522527,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,20220621,0.238095,0.285714,-0.1124,0.005556,-0.810936,0.125217,0.012634,0.657617,0.015679,...,0,0,0,1,0,0,0,0,0,0
2027,20220622,0.227273,0.227273,-1.2702,0.005556,-0.807400,-1.345733,1.613408,0.651895,1.810997,...,0,0,0,1,0,0,0,0,0,0
2028,20220623,0.363636,0.363636,1.7153,0.005556,2.629145,1.459178,2.942254,6.912405,2.129201,...,0,0,0,1,0,0,0,0,0,0
2029,20220624,0.200000,0.300000,1.1716,0.005556,1.323591,0.901009,1.372647,1.751893,0.811817,...,0,0,0,1,0,0,0,0,0,0


### 3.2 回归结果

#### 3.2.1 基准回归

In [17]:
%%stata
ssc install estout

checking estout consistency and verifying not already installed...
installing into /home/ubuntu/ado/plus/...
installation complete.


In [19]:
%%stata -d df_series_ols -force

//描述性统计
//cd Outputs
// outreg2 using Decriptive_statistics.doc, replace sum(log) title(Decriptive statistics)

//时间设定
ge time = _n
tsset time

//VAR回归

foreach var in shareindex_return HIGH    {
   eststo: quietly var `var' img_neg `var'_s, lags(1/5) exog(month_* weekday_*)
   estadd local Month "Yes",replace
   estadd local Weekday "Yes",replace
}

//输出
esttab



. 
. //描述性统计
. //cd Outputs
. // outreg2 using Decriptive_statistics.doc, replace sum(log) title(Decriptive
>  statistics)
. 
. //时间设定
. ge time = _n

. tsset time

Time variable: time, 1 to 2031
        Delta: 1 unit

. 
. //VAR回归
. 
. foreach var in shareindex_return HIGH    {
  2.    eststo: quietly var `var' img_neg `var'_s, lags(1/5) exog(month_* weekd
> ay_*)
  3.    estadd local Month "Yes",replace
  4.    estadd local Weekday "Yes",replace
  5. }
(est4 stored)

added macro:
              e(Month) : "Yes"

added macro:
            e(Weekday) : "Yes"
(est5 stored)

added macro:
              e(Month) : "Yes"

added macro:
            e(Weekday) : "Yes"

. 
. //输出
. esttab

-------------------------------------------------------------------------------
> -------------
                      (1)             (2)             (3)             (4)      
>        (5)   
             shareindex~n            HIGH             LOW    shareindex~n      
>       HIGH   
-----------------------

In [9]:
# r_result['r(table)'].T[:,[0,]]
esttab, keep(mpg:L*.price) star( * 0.1 ** 0.05 *** 0.01) stats( Month Weekday  r2_1, fmt(%3s %3s  %12.4f)) b(%12.4f)  title("Table1 Var") mtitle("OLS" "OLS" "FE" "FE")  nogap obslast  mgroups("Number of deaths" "Number of divorces", pattern(1 0 1 0) span  )

In [10]:
# beta_matrix = (
#     pd.DataFrame(r_result['r(table)'][[0, 3], 5:10].T,
#     columns=pd.MultiIndex.from_tuples([('399300.SZ', 'coff'), ('399300.SZ', 'p_value')]))
# )
# r_result['r(table)']

#### 3.2.2 分组回归

高异质波动率

低异质波动率

回归系数矩阵

## 4.按照观测窗口构造投资策略

In [11]:
def cal_return(df, ma):
    df[f'img_neg_m{ma}'] = (df['img_neg'].rolling(ma).mean())

    # 历史均值
    df['sell_signal'] = df['img_neg'] >= df[f'img_neg_m{ma}']
    df['sell_signal'] = df['sell_signal'].shift(1)

    # 高于均值投资
    df['img_return'] = np.where(df['sell_signal'], -1*(df['sell_signal']*df['HIGH']), df['shareindex_return'])

    # 去掉空行
    df.dropna(axis=0, inplace=True)

    # 换算
    df['mv_shareindex'] = ((df['shareindex_return'] + 100)/100)
    df['mv_img'] = ((df['img_return'] + 100)/100)
    df['mv_shareindex'] = df['mv_shareindex'].cumprod(axis=0)
    df['mv_img'] = df['mv_img'].cumprod(axis=0)

    return df.rename(columns={'mv_img': f'mv_img_{ma}'})

def start():
    df_in = df_series
    for i in [5, ]:
        df_in = cal_return(df_in, i)

    return df_in

# start()